In [1]:
!pip install git+https://github.com/tensorflow/examples.git
# !pip install tensorflow
# !pip install matplotlib

  Running command git clone -q https://github.com/tensorflow/examples.git 'C:\Users\abhis\AppData\Local\Temp\pip-req-build-t0oaygq9'
You should consider upgrading via the 'C:\Users\abhis\OneDrive\Desktop\CycleGAN\venv\Scripts\python.exe -m pip install --upgrade pip' command.



  Cloning https://github.com/tensorflow/examples.git to c:\users\abhis\appdata\local\temp\pip-req-build-t0oaygq9


In [2]:
import tensorflow as tf

# import tensorflow_datasets as tfds
from tensorflow_examples.models.pix2pix import pix2pix

import os
import time
import matplotlib.pyplot as plt
from IPython.display import clear_output

AUTOTUNE = tf.data.AUTOTUNE

In [7]:
# train_size = int(0.7 * DATASET_SIZE)
# val_size = int(0.15 * DATASET_SIZE)
# test_size = int(0.15 * DATASET_SIZE)

full_dataset = tf.data.TFRecordDataset()
full_dataset
# full_dataset = full_dataset.shuffle(buffer_size)
# train_dataset = full_dataset.take(train_size)
# test_dataset = full_dataset.skip(train_size)
# val_dataset = test_dataset.skip(test_size)
# test_dataset = test_dataset.take(test_size)

<TFRecordDatasetV2 shapes: (), types: tf.string>

In [7]:
directory_photos = './datasets/photo_jpg'
directory_paint = './datasets/monet_jpg'

image_generator = tf.keras.preprocessing.image.ImageDataGenerator(validation_split=0.2)
train_photos = image_generator.flow_from_directory(directory=directory_photos,
                                                     subset='training')
test_photos = image_generator.flow_from_directory(directory=directory_photos,
                                                   subset='validation')

train_paint = image_generator.flow_from_directory(directory=directory_paint,
                                                     subset='training')
test_paint = image_generator.flow_from_directory(directory=directory_paint,
                                                   subset='validation')

Found 5631 images belonging to 1 classes.
Found 1407 images belonging to 1 classes.
Found 240 images belonging to 1 classes.
Found 60 images belonging to 1 classes.


In [8]:
BUFFER_SIZE = 1000
BATCH_SIZE = 1
IMG_WIDTH = 256
IMG_HEIGHT = 256

In [9]:
def random_crop(image):
  cropped_image = tf.image.random_crop(
      image, size=[IMG_HEIGHT, IMG_WIDTH, 3])

  return cropped_image

In [10]:
# normalizing the images to [-1, 1]
def normalize(image):
  image = tf.cast(image, tf.float32)
  image = (image / 127.5) - 1
  return image

In [11]:
def random_jitter(image):
  # resizing to 286 x 286 x 3
  image = tf.image.resize(image, [286, 286],
                          method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

  # randomly cropping to 256 x 256 x 3
  image = random_crop(image)

  # random mirroring
  image = tf.image.random_flip_left_right(image)

  return image

In [12]:
def preprocess_image_train(image, label):
  image = random_jitter(image)
  image = normalize(image)
  return image

In [13]:
def preprocess_image_test(image, label):
  image = normalize(image)
  return image